## Installation and imports




In [ ]:
# install DLOmix with wandb extras
#!python -m pip install dlomix[wandb] >= 0.0.5
#!python -m pip install git+https://github.com/wilhelm-lab/dlomix

In [ ]:
# import necessary packages
import tensorflow as tf

import dlomix
from dlomix import (
    constants, data, eval,
    layers, models, pipelines, reports, utils
)
from dlomix.data import RetentionTimeDataset
from dlomix.models import RetentionTimePredictor
from dlomix.eval import TimeDeltaMetric

# Initialize Weights & Biases


In [ ]:
import wandb
from wandb.keras import WandbCallback
from wandb.keras import WandbMetricsLogger
#import wandb.apis.reports as wr


# Create config

config = {
  "seq_length" : 30,
  "batch_size" : 64,
  "val_ratio" : 0.2,
  "lr" : 0.001,
  "optimizer" : "Adam",
  "loss" : "mse"
}

In [ ]:
# Initialize WANDB

PROJECT = 'Demo_RetentionTimeReport'
RUN = "run_1"
wandb.init(project = PROJECT, name = RUN, config = config)

# Load data

In [ ]:
# load small train dataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/RetentionTime/proteomeTools_train_val.csv'

# create dataset object
rtdata = RetentionTimeDataset(data_source = TRAIN_DATAPATH,
                              seq_length = config["seq_length"],
                              batch_size = config["batch_size"],
                              val_ratio = config["val_ratio"],
                              test = False)

# Initialize report

In [ ]:
# Create a report

from dlomix.reports import RetentionTimeReportRunComparisonWandb
report = RetentionTimeReportRunComparisonWandb(project = "Demo_RetentionTimeReport",
                title = "Comparison of different optimizers",
                description = "Comparison of two optimizers Adam and RMSprop",
                dataset = rtdata)

# Log data to Weights & Biases

In [ ]:
report.log_data()

# Train model

In [ ]:
# create retention time predictor
model = RetentionTimePredictor(seq_length = config["seq_length"])


In [ ]:
# create the optimizer object
optimizer = tf.keras.optimizers.Adam(learning_rate = config["lr"])

# compile the model with the optimizer and the metrics we want to use
model.compile(optimizer = optimizer,
              loss = config["loss"],
              metrics=['mean_absolute_error', TimeDeltaMetric()])

In [ ]:
# train the model
history = model.fit(rtdata.train_data,
                    validation_data=rtdata.val_data,
                    epochs=2,
                    callbacks=[WandbMetricsLogger(log_freq = "batch")])

In [ ]:
# log model data to wandb
report.log_model_data(model)

In [ ]:
# finish wandb run
wandb.finish()

# Create Report

In [ ]:
report.create_report()